In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import warnings
import librosa
warnings.filterwarnings('ignore')

In [2]:
from scipy.signal import resample

In [3]:
label = pd.read_csv('../hf_round2_train.txt', sep='\t', names = list(range(0,12)))
label.columns = ['id','age','gender','f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10','f11']
arrythmia = pd.read_csv('../hf_round2_arrythmia.txt',header=None,sep="\t")#读入心电异常事件列表

In [4]:
arrythmia.shape, arrythmia[0].nunique()

((34, 1), 34)

In [5]:
label_map = dict(zip(arrythmia[0].unique(), range(arrythmia.shape[0])))

In [6]:
y_label = np.zeros((label.shape[0], arrythmia.shape[0])).astype(int)
for i in range(label.shape[0]):
    dis = list(label.iloc[i, 3:].unique())
    dis.pop()
    for d in dis:
        if type(d) == str:
            y_label[i, label_map[d]] = 1
y_label.shape

(20036, 34)

In [7]:
class ToTensor(object):
    """
    convert ndarrays in sample to Tensors.
    return:
        feat(torch.FloatTensor)
        label(torch.LongTensor of size batch_size x 1)
    """
    def __call__(self, data):
        data = torch.from_numpy(data).type(torch.FloatTensor)
        return data

In [8]:
import torch
import torch.nn.functional as F
from torch.nn import MaxPool2d
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
class FocalLoss(nn.Module):
    def __init__(self, gamma=2):
        super().__init__()
        self.gamma = 2
        
    def forward(self, input, target):
        assert target.size() == input.size()
        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + ((-max_val).exp() + (-input - max_val).exp()).log()
        invporbs = F.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invporbs * self.gamma).exp() * loss
        return loss.sum(dim=1).mean()

In [10]:
data = pd.read_csv('../hf_round2_train/100001.txt', sep=' ')
data['III'] = data['II']-data['I']
data['aVR']=-(data['II']+data['I'])/2
data['aVL']=(data['I']-data['II'])/2
data['aVF']=(data['II']-data['I'])/2
data = data.astype('float').values

In [11]:
label.head()

,id,age,gender,f3,f4,f5,f6,f7,f8,f9,f10,f11
0,100001.txt,47.0,MALE,窦性心律,T波改变,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003.txt,40.0,MALE,窦性心动过速,T波改变,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100005.txt,42.0,MALE,窦性心律,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100007.txt,21.0,FEMALE,窦性心动过速,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100009.txt,49.0,MALE,窦性心动过速,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
label['age'] = label['age'].fillna(0.0)

In [13]:
label['gender'] = label['gender'].fillna('MISS')

In [14]:
import scipy

In [15]:
from multiprocessing import Pool
from scipy import stats

In [16]:
#with Pool(32) as p:
 #   Feats = p.map(Get_Feat, list(label['id']))

In [17]:
from biosppy.signals import ecg
from pyentrp import entropy as ent
import pywt

In [18]:
def WTfilt_1d(sig):
    """
    # 使用小波变换对单导联ECG滤波
    # 参考：Martis R J, Acharya U R, Min L C. ECG beat classification using PCA, LDA, ICA and discrete
    wavelet transform[J].Biomedical Signal Processing and Control, 2013, 8(5): 437-448.
    :param sig: 1-D numpy Array，单导联ECG
    :return: 1-D numpy Array，滤波后信号
    """
    coeffs = pywt.wavedec(sig, 'db6', level=9)
    coeffs[-1] = np.zeros(len(coeffs[-1]))
    coeffs[-2] = np.zeros(len(coeffs[-2]))
    coeffs[0] = np.zeros(len(coeffs[0]))
    sig_filt = pywt.waverec(coeffs, 'db6')
    return sig_filt

In [19]:
class ManFeat_HRV(object):
    """
        针对一条记录的HRV特征提取， 以II导联为基准
    """
    FEAT_DIMENSION = 9

    def __init__(self, sig, fs=250.0):
        assert len(sig.shape) == 1, 'The signal must be 1-dimension.'
        assert sig.shape[0] >= fs * 6, 'The signal must >= 6 seconds.'
        self.sig = WTfilt_1d(sig)
        self.fs = fs
        self.rpeaks, = ecg.hamilton_segmenter(signal=self.sig, sampling_rate=self.fs)
        self.rpeaks, = ecg.correct_rpeaks(signal=self.sig, rpeaks=self.rpeaks,
                                         sampling_rate=self.fs)
        self.RR_intervals = np.diff(self.rpeaks)
        self.dRR = np.diff(self.RR_intervals)
    
    def __get_sdnn(self):  # 计算RR间期标准差
        return np.array([np.std(self.RR_intervals)])

    def __get_maxRR(self):  # 计算最大RR间期
        return np.array([np.max(self.RR_intervals)])

    def __get_minRR(self):  # 计算最小RR间期
        return np.array([np.min(self.RR_intervals)])

    def __get_meanRR(self):  # 计算平均RR间期
        return np.array([np.mean(self.RR_intervals)])

    def __get_Rdensity(self):  # 计算R波密度
        return np.array([(self.RR_intervals.shape[0] + 1) 
                         / self.sig.shape[0] * self.fs])

    def __get_pNN50(self):  # 计算pNN50
        return np.array([self.dRR[self.dRR >= self.fs*0.05].shape[0] 
                         / self.RR_intervals.shape[0]])

    def __get_RMSSD(self):  # 计算RMSSD
        return np.array([np.sqrt(np.mean(self.dRR*self.dRR))])
    
    def __get_SampEn(self):  # 计算RR间期采样熵
        sampEn = ent.sample_entropy(self.RR_intervals, 
                                  2, 0.2 * np.std(self.RR_intervals))
        for i in range(len(sampEn)):
            if np.isnan(sampEn[i]):
                sampEn[i] = -2
            if np.isinf(sampEn[i]):
                sampEn[i] = -1
        return sampEn

    def extract_features(self):  # 提取HRV所有特征
        if len(self.RR_intervals) == 0 or len(self.dRR) == 0:
            features =  np.zeros((ManFeat_HRV.FEAT_DIMENSION,))
        else:
            features = np.concatenate((self.__get_sdnn(),
                    self.__get_maxRR(),
                    self.__get_minRR(),
                    self.__get_meanRR(),
                    self.__get_Rdensity(),
                    self.__get_pNN50(),
                    self.__get_RMSSD(),
                    self.__get_SampEn(),
                    ))
        assert features.shape[0] == ManFeat_HRV.FEAT_DIMENSION
        return features

In [20]:
def get_RR(data):
    result = []
    for i in range(12):
        arr = data[:,i]
        s = ManFeat_HRV(arr, fs=250)
        fea = s.extract_features()
        result.append(fea)
    result = np.concatenate(result)
    return result

In [21]:
def RR_Feat(Id):
    data = pd.read_csv('../hf_round2_train/'+Id, sep=' ')
    data['III'] = data['II']-data['I']
    data['aVR']=-(data['II']+data['I'])/2
    data['aVL']=(data['I']-data['II'])/2
    data['aVF']=(data['II']-data['I'])/2
    data = data.astype('float').values
    data = resample(data, 2500)
    feas = get_RR(data)
    return feas

In [22]:
with Pool(12) as p:
    Feat_RR = p.map(RR_Feat, list(label['id']))
Feat_RR = np.stack(Feat_RR)
Feat_RR.shape

(20036, 108)

In [23]:
age_mean = label['age'].mean()

In [24]:
age_std = label['age'].std()

In [25]:
class Data(Dataset):
    def __init__(self, fname, mode,  y, feat, transform=None):
        self.fname = fname
        self.mode = mode
        self.transform = transform
        self.y = y
        self.feat = feat
        
    def __len__(self):
        return self.fname.shape[0]
    
    def __getitem__(self, idx):
        filename = self.fname['id'].iloc[idx]
        age = self.fname['age'].iloc[idx]
        age = (age -age_mean)/age_std
        gender = self.fname['gender'].iloc[idx]
        if self.mode=='train':
            filepath = os.path.join('../hf_round2_train/', filename.split('.')[0]+'.txt')
        data = pd.read_csv(filepath, sep=' ')
        data['III'] = data['II']-data['I']
        data['aVR']=-(data['II']+data['I'])/2
        data['aVL']=(data['I']-data['II'])/2
        data['aVF']=(data['II']-data['I'])/2
        data = data.astype('float').values
        data = resample(data, 2500).transpose()
        fea = self.feat[idx].reshape(1,-1)
        fea2 = np.zeros((4,))
        fea2[0] = age
        if gender == 'MALE':
            fea2[1] = 1
        if gender == 'FEMALE':
            fea2[2] = 1
        if gender == 'MISS':
            fea2[3] = 1
        if self.transform is not None:
            data = self.transform(data)
            fea = self.transform(fea)
            fea2 = self.transform(fea2)
        if self.mode == 'train':
            L = self.y[idx]
            return data, fea, fea2, L
        else:
            return img

In [26]:
s = Data(label, 'train',  y_label,Feat_RR)

In [27]:
s[0][0].shape, s[0][1].shape, s[0][2].shape, s[0][3].shape

((12, 2500), (1, 108), (4,), (34,))

In [28]:
#from resnet import resnet34, resnet50, resnet18
#from densenet121 import densenet121
#from densenet import densenet121
#from img_net import img_net
from senet import se_resnet50, se_resnext50_32x4d, se_resnet101
#from resxnet32 import resnext101_32x4d_features
#from resnet_no_head import resnetm34
#from eff import EfficientNet

In [29]:
class Model(nn.Module):
    def __init__(self, base):
        super(Model, self).__init__()
        self.base = base
        self.bn1 = nn.BatchNorm1d(1)
        self.classifier = nn.Linear(2160, 34)
    def forward(self, x1, x2, x3):
        x1 = self.base(x1)
        x2 = self.bn1(x2)
        x2 = x2.view(x2.size(0), -1)
        x = torch.cat([x1,x2,x3], 1)
        x = self.classifier(x)
        return x

In [30]:
from sklearn.metrics import f1_score

In [31]:
wc = y_label.sum(axis=0)
weight = 1.0/np.log(wc)
weight

array([0.91023923, 0.14235587, 0.36067376, 0.12263167, 0.14235587,
       0.20887764, 0.24044917, 0.25308487, 0.36067376, 0.17680365,
       0.20178267, 0.31066747, 0.15843505, 0.31066747, 0.28126641,
       0.28853901, 0.31892899, 0.16595125, 0.11670455, 0.32351545,
       0.10918041, 0.51389834, 0.17542494, 0.36067376, 0.2969742 ,
       0.28357849, 0.15880302, 0.17393141, 0.14698306, 0.16568687,
       0.11770287, 0.20677041, 0.28126641, 0.24423934])

In [32]:
import warnings
warnings.filterwarnings('ignore')

In [33]:
def train_model(x_train, x_val, train_label,val_label,fold,weight,Model,feat):
    num_epochs = 68
    batch_size = 32
    test_batch_size = 32
    lr = 1e-3
    eta_min = 1e-5
    t_max = 25
    
    num_classes = 34
    
    train_dataset = Data(x_train, 'train',train_label, feat, transform=transforms.Compose([ToTensor()]))
    valid_dataset = Data(x_val, 'train', val_label, feat, transform=transforms.Compose([ToTensor()]))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=20)
    valid_loader = DataLoader(valid_dataset, batch_size=test_batch_size, shuffle=False, num_workers=20)
    model = Model.cuda()
    #criterion = FocalLoss().cuda()
    w = torch.tensor(weight, dtype=torch.float).cuda()
    criterion = nn.MultiLabelSoftMarginLoss(weight=w).cuda()
    val_criterion = nn.BCEWithLogitsLoss().cuda()
    optimizer = Adam(params=model.parameters(), lr=lr, amsgrad=False)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(factor=0.1, mode='max', min_lr=eta_min, patience=3, optimizer=optimizer)
    #scheduler = CosineAnnealingLR(optimizer, T_max=t_max, eta_min=eta_min)
    best_epoch = -1
    best_loss = 0
    mb = range(num_epochs)
    for epoch in mb:
        start_time = time.time()
        model.train()
        avg_loss = 0.

        for x_batch, x1, x2, y_batch in train_loader:
            #x_batch, y_batch, lam = mixup_data(x_batch.cuda(), y_batch.cuda(), alpha=1.0)
            preds = model(x_batch.cuda(),x1.cuda(), x2.cuda())
            loss = criterion(preds, y_batch.cuda().float())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            avg_loss += loss.item() / len(train_loader)

        model.eval()
        valid_preds = np.zeros((len(x_val), num_classes))
        avg_val_loss = 0.
        pred_list = []
        label_list = []
        for i, (x_batch, x1, x2, y_batch) in enumerate(valid_loader):
            preds = model(x_batch.cuda(), x1.cuda(), x2.cuda()).detach()
            loss = criterion(preds, y_batch.cuda().float())

            preds = torch.sigmoid(preds)
            valid_preds[i * test_batch_size: (i+1) * test_batch_size] = preds.cpu().numpy()

            avg_val_loss += loss.item() / len(valid_loader)
            predictions = preds.cpu().numpy()
            label = y_batch.cpu().numpy()
            pred_list.append(predictions)
            label_list.append(label)
        pred_list = np.concatenate(pred_list).reshape(-1)
        label_list = np.concatenate(label_list).reshape(-1)
        lss = []
        for threshold in np.linspace(0, 1, 101):
            s = f1_score(label_list, (pred_list>threshold).astype(int))
            lss.append((threshold, s))
        avg_f1 = sorted(lss, key=lambda x:x[1], reverse=True)[0][1]
        threshold = sorted(lss, key=lambda x:x[1], reverse=True)[0][0]
            
        
        
        scheduler.step(avg_f1)
        #scheduler.step()

        if (epoch + 1) % 1 == 0:
            elapsed = time.time() - start_time
            print('Epoch %d - avg_train_loss: %f  avg_val_loss: %f  avg_f1:%f threshold:%f time: %f s'%(epoch+1, avg_loss, avg_val_loss,avg_f1, threshold, elapsed))
    
        if avg_f1 > best_loss:
            best_epoch = epoch + 1
            best_loss = avg_f1
            torch.save(model.state_dict(), 'weight_best_%s.pt'%str(fold))          
    return {
        'best_epoch': best_epoch,
        'best_loss': best_loss
    }

In [34]:
from sklearn.model_selection import KFold
import time
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
mskf = MultilabelStratifiedKFold(n_splits=5, random_state=2018)
kf = KFold(n_splits=5, random_state=2018, shuffle=True)
for foldNum, (train_split, val_split) in enumerate(mskf.split(label, y_label)):
    end = time.time()
    # split the dataset for cross-validation
    train_fname = label.iloc[train_split]
    train_label = y_label[train_split]
    val_fname = label.iloc[val_split]
    val_label = y_label[val_split]
    print(train_fname.shape,train_label.shape)
    print("Fold {0}, Train samples:{1}, val samples:{2}".format(foldNum, len(train_fname), len(val_fname)))
    #result = train_model(train_fname, val_fname, train_label, val_label,foldNum, weight, Model(se_resnext50_32x4d(num_classes=34, pretrained=None)),Feat_RR )
    #result = train_model(train_fname, val_fname, train_label, val_label,foldNum, weight, EfficientNet(1,1,num_classes=34))
    #print(result)

(16027, 12) (16027, 34)
Fold 0, Train samples:16027, val samples:4009
(16031, 12) (16031, 34)
Fold 1, Train samples:16031, val samples:4005
(16031, 12) (16031, 34)
Fold 2, Train samples:16031, val samples:4005
(16026, 12) (16026, 34)
Fold 3, Train samples:16026, val samples:4010
(16029, 12) (16029, 34)
Fold 4, Train samples:16029, val samples:4007


## predict

In [35]:
class TestData(Dataset):
    def __init__(self, fname,mode, feat,  transform=None):
        self.fname = fname
        self.transform = transform
        self.mode=mode
        self.feat = feat
        
    def __len__(self):
        return self.fname.shape[0]
    
    def __getitem__(self, idx):
        filename = self.fname['id'].iloc[idx]
        age = self.fname['age'].iloc[idx]
        age = (age -age_mean)/age_std
        gender = self.fname['gender'].iloc[idx]
        if self.mode == 'train':
            filepath = os.path.join('../hf_round2_train/', filename.split('.')[0]+'.txt')
        else:
            filepath = os.path.join('tcdata/hf_round2_testA/', filename.split('.')[0]+'.txt')
        data = pd.read_csv(filepath, sep=' ')
        data['III'] = data['II']-data['I']
        data['aVR']=-(data['II']+data['I'])/2
        data['aVL']=(data['I']-data['II'])/2
        data['aVF']=(data['II']-data['I'])/2
        data = data.astype('float').values
        data = resample(data, 2500).transpose()
        fea = self.feat[idx].reshape(1,-1)
        fea2 = np.zeros((4,))
        fea2[0] = age
        if gender == 'MALE':
            fea2[1] = 1
        if gender == 'FEMALE':
            fea2[2] = 1
        if gender == 'MISS':
            fea2[3] = 1
        if self.transform is not None:
            data = self.transform(data)
            fea = self.transform(fea)
            fea2 = self.transform(fea2)
        return data, fea,fea2, filename

In [36]:
#model_conv = resnet34(pretrained=False)
#model_conv.conv1 = nn.Conv2d(12, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False)
#model_conv = nn.Sequential(*([nn.BatchNorm2d(12)]+list(model_conv.children())[:-1]))
#model = Model(resnetm34(), model_conv).cuda()
model = Model(se_resnext50_32x4d(num_classes=34, pretrained=None)).cuda()
valres = []
for foldNum, (train_split, val_split) in enumerate(mskf.split(label, y_label)):
    end = time.time()
    # split the dataset for cross-validation
    val_fname = label.iloc[val_split]
    val_label = y_label[val_split]
    valData = TestData(val_fname, 'train',Feat_RR, transform=transforms.Compose([ToTensor()]))
    valloader = DataLoader(valData, batch_size=64, shuffle=False)
    model.load_state_dict(torch.load('weight_best_%s.pt'% str(foldNum)))
    model.cuda()
    model.eval()

    val_outputs = []
    val_fnames = []
    for images, fea, fea2, fnames in valloader:
        preds = torch.sigmoid(model(images.cuda(), fea.cuda(), fea2.cuda()).detach())
        val_outputs.append(preds.cpu().numpy())
        val_fnames.extend(fnames)

    val_preds = pd.DataFrame(data=np.concatenate(val_outputs),
                                  index=val_fnames,
                                  columns=map(str, range(34)))
    val_preds = val_preds.groupby(level=0).mean()
    valres.append(val_preds)
    print(1)
val_predsss = pd.concat([valres[0].reset_index(),valres[1].reset_index(),valres[2].reset_index(),valres[3].reset_index(),valres[4].reset_index()])

1
1
1
1
1


In [37]:
y_label.shape,val_predsss.shape

((20036, 34), (20036, 35))

In [38]:
resall = val_predsss.rename(columns={'index':'id'})
kkksksk = pd.merge(label[['id']], resall, how='left', on='id')
from sklearn.metrics import f1_score
pred_pro = kkksksk.drop('id', axis=1).values
pred_pro = pred_pro.reshape(-1)
lss = []
for i in np.linspace(0, 1, 101):
    s = f1_score(y_label.reshape(-1), (pred_pro>i).astype(int))
    lss.append((i, s))

In [40]:
kkksksk.to_csv('../se_resnet50_32x4d_val.csv')

In [39]:
sorted(lss, key=lambda x:x[1], reverse=True)[0]

(0.53, 0.9187668383180438)

In [ ]:
ks2 = kkksksk.copy()
map2 = dict(zip(range(34), [0.49]*34))
for j in range(34):
    res = []
    for th in np.linspace(0, 1, 101):
        for i in range(34):
            if i == j:
                ks2[str(i)] = (kkksksk[str(i)] > th).astype(int)
            else:
                ks2[str(i)] = (kkksksk[str(i)] > map2[i]).astype(int)
        s = f1_score(y_label.reshape(-1), ks2.drop('id', axis=1).values.reshape(-1))
        res.append((th, s))
    map2[j] = sorted(res, key=lambda x:x[1], reverse=True)[0][0]

In [ ]:
map2 = {0: 0.1,1: 0.48,2: 0.88,3: 0.43,4: 0.44,5: 0.5700000000000001,6: 0.59,7: 0.86,8: 0.99,9: 0.67,10: 0.35000000000000003,
 11: 0.8200000000000001,12: 0.5700000000000001,13: 0.8200000000000001,14: 0.76,15: 0.86,16: 0.87,17: 0.48,18: 0.54,19: 0.96,
 20: 0.48,21: 0.6,22: 0.58,23: 0.47000000000000003,24: 0.49,25: 0.96,26: 0.55,27: 0.56,28: 0.65,29: 0.47000000000000003,30: 0.47000000000000003,
 31: 0.6900000000000001,32: 0.98,33: 0.81}
map2

In [ ]:
for i in range(34):
    ks2[str(i)] = (kkksksk[str(i)] > map2[i]).astype(int)

In [ ]:
f1_score(y_label.reshape(-1), ks2.drop('id', axis=1).values.reshape(-1))

In [ ]:
sorted(lss, key=lambda x:x[1], reverse=True)[0]

In [ ]:
class TestData(Dataset):
    def __init__(self, fname,mode,transform=None):
        self.fname = fname
        self.transform = transform
        self.mode=mode
        
    def __len__(self):
        return self.fname.shape[0]
    
    def __getitem__(self, idx):
        filename = self.fname['id'].iloc[idx]
        if self.mode == 'train':
            filepath = os.path.join('hf_round2_train/', filename.split('.')[0]+'.txt')
        else:
            filepath = os.path.join('tcdata/hf_round2_testA/', filename.split('.')[0]+'.txt')
        data = pd.read_csv(filepath, sep=' ')
        data['III'] = data['II']-data['I']
        data['aVR']=-(data['II']+data['I'])/2
        data['aVL']=(data['I']-data['II'])/2
        data['aVF']=(data['II']-data['I'])/2
        data = data.astype('float').values
        data = resample(data, 2500)
        #mean = np.mean(data)
        #std = np.std(data)
        #data = (data -mean)/(std+1e-8)
        # if random.random() > 0.5:
        #    data = np.flipud(data).copy()
        #if random.random() > 0.5:
        #    start = random.randint(0, 2250)
        #    data[start:start+250,:] = 0
        img = data.transpose()
        if self.transform is not None:
            img = self.transform(img)
        return img, filename

In [ ]:
model = resnet34().cuda()
valres = []
for foldNum, (train_split, val_split) in enumerate(mskf.split(label, y_label)):
    end = time.time()
    # split the dataset for cross-validation
    val_fname = label.iloc[val_split]
    val_label = y_label[val_split]
    valData = TestData(val_fname, 'train',transform=transforms.Compose([ToTensor()]))
    valloader = DataLoader(valData, batch_size=64, shuffle=False)
    model.load_state_dict(torch.load('9211/weight_best_%s.pt'% str(foldNum)))
    model.cuda()
    model.eval()

    val_outputs = []
    val_fnames = []
    for images, fnames in valloader:
        preds = torch.sigmoid(model(images.cuda()).detach())
        val_outputs.append(preds.cpu().numpy())
        val_fnames.extend(fnames)

    val_preds = pd.DataFrame(data=np.concatenate(val_outputs),
                                  index=val_fnames,
                                  columns=map(str, range(34)))
    val_preds = val_preds.groupby(level=0).mean()
    valres.append(val_preds)
    print(1)
val_predsss2 = pd.concat([valres[0].reset_index(),valres[1].reset_index(),valres[2].reset_index(),valres[3].reset_index(),valres[4].reset_index()])

In [ ]:
total = pd.concat([val_predsss[['index']], (val_predsss2.drop('index', axis=1) + val_predsss.drop('index', axis=1))/2], axis=1)

In [ ]:
resall = total.rename(columns={'index':'id'})
kkksksk = pd.merge(label[['id']], resall, how='left', on='id')
from sklearn.metrics import f1_score
pred_pro = kkksksk.drop('id', axis=1).values
pred_pro = pred_pro.reshape(-1)
lss = []
for i in np.linspace(0, 1, 101):
    s = f1_score(y_label.reshape(-1), (pred_pro>i).astype(int))
    lss.append((i, s))

In [ ]:
sorted(lss, key=lambda x:x[1], reverse=True)[0]

In [ ]:
val_predsss = pd.concat([valres[0].reset_index(),valres[1].reset_index(),valres[2].reset_index(),valres[3].reset_index(),valres[4].reset_index()])

In [ ]:
testpredsss = (testres[0]+testres[1]+testres[2]+testres[3]+testres[4])/5

In [ ]:
testresult = testpredsss.reset_index()

In [ ]:
val_predsss = val_predsss.rename(columns={'index':'id'})

In [ ]:
kkksksk = pd.merge(label[['id']], val_predsss, how='left', on='id')

In [ ]:
label_map_reverse = {v:k for (k,v) in label_map.items()}

In [ ]:
with open('hf_round1_subB_noDup_rename.txt', 'r') as f:
    su = f.readlines()
    su = [x[:-1] for x in su]

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
pred_pro = kkksksk.drop('id', axis=1).values
pred_pro = pred_pro.reshape(-1)

In [ ]:
lss = []
for i in np.linspace(0, 1, 101):
    s = f1_score(y_label.reshape(-1), (pred_pro>i).astype(int))
    lss.append((i, s))

In [ ]:
sorted(lss, key=lambda x:x[1], reverse=True)[0]

In [ ]:
kkksksk.to_csv('ensemble/val_pred_dense121_baseline_2_dense_has_A.csv', index=False)

In [ ]:
testresult.to_csv('ensemble/testB_pred_dense121_baseline_2_dense_has_B.csv', index=False)

In [ ]:
test2 = (testres[0]+testres[1]+testres[2]+testres[3]+testres[4])/5

In [ ]:
test2 = (test2 > 0.48).astype(int)

In [ ]:
test2 = test2.reset_index().rename(columns={'index':'id'})
kkk2 = pd.merge(subB, test2, on='id', how='left')

In [ ]:
with open('subB.txt', 'w') as f:
    for i in range(kkk2.shape[0]):
        data = kkk2.iloc[i]
        res = su[i]
        for j in range(55):
            if data[str(j)] == 1:
                res += '\t'
                res += label_map_reverse[j]
        res += '\n'
        f.write(res)

In [ ]:
!wc -l subB.txt